# Utilizing RAG Framework for a Business-Focused QA Bot

## Step 1 : Installing Dependencies

---



In [ ]:
!pip install pyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.0 MB/s eta 0:00:00


In [ ]:
!pip install groq
!pip install pinecone-client
!pip install firecrawl-py
!pip install google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 7.4 MB/s eta 0:00:00


## Step 2 : Collecting Data from a Website to Supply as Knowledge to the Chatbot

In [ ]:
from firecrawl import FirecrawlApp
import getpass
from google.colab import userdata

# Access the environment variable using os.environ.get()
app = FirecrawlApp(api_key=userdata.get("FIRECRAWL_API_KEY") or getpass.getpass("Enter your Firecrawl API key: "))

crawl_result = app.crawl_url('https://openai.com/', {'crawlerOptions': {'excludes': ['blog/*']}})

# Get the markdown
#for result in crawl_result:
#    print(result['markdown'])



In [ ]:
print('No.of URLs and sub URLs processed: ',len(crawl_result))
print(crawl_result[2])

No.of URLs and sub URLs processed:  61
{'content': '[Skip to main content](#main)\n\n[](/)\n\n*   Research\n*   Products\n*   Safety\n*   Company\n\nChatGPT on your desktop\n=======================\n\nChat about email, screenshots, files, and anything on your screen.\n\n[Download](https://persistent.oaistatic.com/sidekick/public/ChatGPT_Desktop_public_latest.dmg)\n\nAvailable now on macOS.\\*\n\n### Seamlessly integrates with how you work, write, and create\n\n###### Faster access to ChatGPT\n\nOption + Space opens ChatGPT from any screen on your desktop.\n\n![macapp launcher](https://images.ctfassets.net/kftzwdyauwt9/cGntK6W80RyEWtLMRkY7a/f19e63f01c934e1247f988739dd99d64/macapp_shortcut.png?w=3840&q=90&fm=webp)\n\nTake screenshotsUpload filesAsk anythingShare imagesSearch conversations\n\nTake screenshotsUpload filesAsk anythingShare imagesSearch conversations\n\nYour browser does not support the video tag.\n\nYour browser does not support the video tag.\n\nYour browser does not suppo

## Step 3 : Chuncking

In [ ]:
from typing import List, Dict

def chunk_text(crawl_results: List[Dict], chunk_size: int = 512) -> List[Dict]:
    """Chunks a given text into specified sizes."""
    # Extract the content from the crawl results
    text = " ".join([result.get('content', '') for result in crawl_results])

    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size):
        chunks.append({
            "id": f"chunk-{i // chunk_size}",
            "content": " ".join(words[i:i + chunk_size])
        })
    return chunks


data = chunk_text(crawl_result) # Pass the list of results directly
print(data[2])
print('Total no.of chunks the crawl_result is  divided into :',len(data))

{'id': 'chunk-2', 'content': 'in ChatGPT\\ \\ ![ChatGPT Charts Blog Hero](https://images.ctfassets.net/kftzwdyauwt9/3vjzQxExAEnYGvBSiwT1zK/d76028c004dbc7e9830e541cced6224f/ChatGPT_Charts_Blog_Hero.png?w=3840&q=90&fm=webp)](/index/improvements-to-data-analysis-in-chatgpt/) [News\\ \\ ChatGPT can now see, hear, and speak\\ \\ Your browser does not support the video tag.](/index/chatgpt-can-now-see-hear-and-speak/) [News\\ \\ ChatGPT can now see, hear, and speak\\ \\ Your browser does not support the video tag.](/index/chatgpt-can-now-see-hear-and-speak/) [News\\ \\ News\\ \\ News\\ \\ Introducing the GPT Store\\ \\ Introducing the GPT Store\\ \\ Introducing the GPT Store\\ \\ ![introducing the gpt store](https://images.ctfassets.net/kftzwdyauwt9/3oQ0LwB3yg3R5isshaaJKi/bf1e997adf17012fceea0feb29d35bf4/introducing_the_gpt_store.jpg?w=3840&q=90&fm=webp)](/index/introducing-the-gpt-store/) [News\\ \\ News\\ \\ News\\ \\ Introducing the GPT Store\\ \\ Introducing the GPT Store\\ \\ Introducin

## Step 4 : Creating a Pinecone Vector Database

In [ ]:
from pinecone import ServerlessSpec
from pinecone import Pinecone
import getpass
from google.colab import userdata




pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY") or getpass.getpass("Enter your Pinecone API key: "))

index_name = "qa-rag" #qa-rag is the Pinecone Index Name


existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=768,
        metric='cosine',
        spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  )

    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

## Step 5: Generate Embeddings for Vector Storage

In [ ]:
import os
import getpass
import google.generativeai as genai
from google.colab import userdata

GEMINI_API_KEY =userdata.get("GEMINI_API_KEY")  or getpass.getpass("Enter your Gemini API key: ")
genai.configure(api_key=GEMINI_API_KEY)

def generate_embeddings(texts: List[str]) -> List[List[float]]:
    """Generates embeddings for a list of text chunks using Gemini."""
    embeddings = []
    for text in texts:
        result = genai.embed_content(
            model="models/text-embedding-004",  # Choose an appropriate embedding model
            content=text,
            task_type="retrieval_document",
            title="PDF Chunk Embedding"
        )
        embeddings.append(result['embedding'])
    return embeddings

# Example usage
embeddings = generate_embeddings([chunk["content"] for chunk in data])
print(embeddings[0][:10])  # Print the first 10 dimensions of the first embedding
print(len(embeddings))
#print(embeddings)

Enter your Gemini API key: ··········
[0.011746227, 0.01827689, -0.012091766, 0.002370531, 0.09047623, 0.03713244, 0.06310191, -0.017052429, 0.03160209, 0.019957732]
203


## Step 6: Storing Embeddings into Pinecone Vector Database

In [ ]:
from pinecone import Pinecone, ServerlessSpec
import time
from google.colab import userdata

api_key = userdata.get("PINECONE_API_KEY") or getpass.getpass("Enter your Pinecone API key: ")
pc = Pinecone(api_key=api_key)

index_name = "qa-rag"  # Choose a name for your index
dims = len(embeddings[0])  # Get the dimensionality from Gemini embeddings

spec = ServerlessSpec(cloud="aws", region="us-west-1")

if index_name not in [info['name'] for info in pc.list_indexes()]:
    # Use the correct dimensionality when creating the index
    pc.create_index(index_name, dimension=dims, metric='cosine', spec=spec)  # Set dimension to dims
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

index = pc.Index(index_name)
time.sleep(1)

# Upsert Embeddings to Pinecone
batch_size = 32
for i in range(0, len(data), batch_size):
    i_end = min(len(data), i + batch_size)
    batch_data = data[i:i_end]
    batch_embeddings = embeddings[i:i_end]
    to_upsert = [(chunk["id"], embedding, {"content": chunk["content"]})
                  for chunk, embedding in zip(batch_data, batch_embeddings)]
    index.upsert(vectors=to_upsert)

## Step 7: Retrieving Data From Pinecone

In [ ]:
from typing import List

def get_docs(query: str, top_k: int = 30) -> List[str]:  # 30 indicates top 30 matched Results
    """Retrieves relevant documents from Pinecone."""
    query_embedding = generate_embeddings([query])[0]  # Generate embedding for the query
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)
    return [x["metadata"]["content"] for x in results["matches"]]

def generate_embeddings(texts: List[str]) -> List[List[float]]:
    """Generates embeddings for a list of text chunks using Gemini."""
    embeddings = []
    for text in texts:
        result = genai.embed_content(
            model="models/text-embedding-004",  # Choose an appropriate embedding model
            content=text,
            task_type="retrieval_document",
            title="PDF Chunk Embedding"
        )
        embeddings.append(result['embedding'])
    return embeddings

user_query = input('Enter Your Query :\n ')
retrieved_docs = get_docs(user_query)
print(retrieved_docs)
print(len(retrieved_docs))

Enter Your Query :
 what is openai
['and reporting vulnerabilities. Our research * [Overview](/research/) * [Index](/news/research/) Latest advancements * [GPT-4](/index/gpt-4/) * [GPT-4o mini](/index/gpt-4o-mini-advancing-cost-efficient-intelligence/) * [DALL·E 3](/index/dall-e-3/) * [Sora](/index/sora/) ChatGPT * [For Everyone](/chatgpt/) * [For Teams](/chatgpt/team/) * [For Enterprises](/chatgpt/enterprise/) * [ChatGPT login(opens in a new window)](https://chatgpt.com/) * [Download](/chatgpt/download/) API * [Platform overview](/api/) * [Pricing](/api/pricing/) * [Documentation(opens in a new window)](https://platform.openai.com/docs/introduction) * [API login(opens in a new window)](https://platform.openai.com/login?launch) Explore more * [OpenAI for business](/business/) * [Stories](/news/stories/) Safety overview * [Safety overview](/safety/) * [Safety standards](/safety-standards/) Teams * [Safety Systems](/safety-systems/) * [Preparedness](/preparedness/) * [Superalignment](/su

## Step 8: Refine the Findings with LLM

In [ ]:

import getpass
from typing import List, Dict, Any
import google.generativeai as genai
from google.colab import userdata

safe = [
        {
            "category": "HARM_CATEGORY_DANGEROUS",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_HARASSMENT",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_HATE_SPEECH",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
            "threshold": "BLOCK_NONE",
        },
    ]


# Generate a response based on the user query and context documents
def generate_response(query: str, docs: List[str],safe: List[str]) -> str:
    """Generates a response using Google Gemini."""
    genai.configure(api_key=userdata.get('GEMINI_API_KEY') or getpass.getpass("Enter your Gemini API key: "))
    model = genai.GenerativeModel('gemini-1.5-flash')  # Gemini Flash Model is the Fastest model of Gemini Faminly and performes well as other Gemini Models
    context = "\n---\n".join(docs)
    prompt = (
        "You are a helpful AI assistant. Don't refuse to answer any question of mine. "
        "Answer the question in detail using the provided context only. "
        "If you don't know the answer, just say that you don't know; don't try to make up an answer.\n\n"
        f"CONTEXT:\n{context}\n\n"
        f"Question: {query}"
        "You are a Helpfull ChatBot answer the Questions as a chatBot would and while answering Don't use words like According to the context or something similar to that just provide me with the answer if the Context Dosen't Provide any answer Just say I am unaware of the Information you are requesting and provide some relevent Information for the Query"
    )

    response = model.generate_content(prompt, safety_settings=safe)
    return response.text



user_query = input('Enter Your Query :\n ')
retrieved_docs = get_docs(user_query)
answer = generate_response(user_query, retrieved_docs,safe)
print('Answer is :',answer)


Enter Your Query :
 is the chatGpt Desktop app is available for Windows Users
Answer is : The ChatGPT desktop app is currently only available for macOS 14+ with Apple Silicon (M1 or better). It's coming to Windows later this year! 



In [ ]:
user_query = input('Enter Your Query :\n ')
retrieved_docs = get_docs(user_query)
answer = generate_response(user_query, retrieved_docs,safe)
print('Answer is :',answer)

Enter Your Query :
 what are the products of Openai
Answer is : OpenAI's products include:

* **ChatGPT:** A conversational AI chatbot for everyone, teams, and enterprises. 
* **GPT-4:** The latest version of OpenAI's large language model.
* **GPT-4o mini:** A cost-efficient version of GPT-4.
* **DALL·E 3:** A powerful AI system for generating images from text descriptions.
* **Sora:** An AI model that can generate videos from text prompts.

OpenAI also offers an API that allows developers to access and use their models. They also have a business division that focuses on providing AI solutions to businesses.



In [ ]:
from groq import Groq
from google.colab import userdata

GROQ_API_KEY = api_key=userdata.get("GROQ_API_KEY") or getpass.getpass("Enter your Groq API key: ")
groq_client = Groq(api_key=GROQ_API_KEY)

def get_docs(query: str, top_k: int = 6) -> List[str]:  # 6 referes to the top 6 matched records
    """Retrieves relevant documents from Pinecone."""
    query_embedding = generate_embeddings([query])[0]  # Generate embedding for the query
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)
    return [x["metadata"]["content"] for x in results["matches"]]

def generate_response(query: str, docs: List[str]) -> str:
    """Generates a response using Groq's LLaMa 2 model."""
    system_message = (
        "You are a helpful AI assistant. Don't refuse to answer any question of mine. "
        "Answer the question in detail using the provided context only. "
        "If you don't know the answer, just say that you don't know; don't try to make up an answer.\n\n"
        f"CONTEXT:\n{''.join(docs)}\n\n"
        f"Question: {query}"
    )
    messages = [
        {"role": "system", "content": system_message},
        {"role": "assistant" ,"content": "If the answer is not Mentioned in the provided Context Then Do not mention that it is not provided in the context in the rsponse"},
         {"role": "assistant", "content": "You are a Helpfull ChatBot answer the Questions as a chatBot would and while answering Don't use words like According to the context or something similar to that just provide me with the answer if the Context Dosen't Provide any answer Just say I am unaware of the Information you are requesting and provide some relevent Information for the Query"}
    ]
    response = groq_client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        messages=messages
    )
    return response.choices[0].message.content



user_query = input('Enter Your Query: ')
retrieved_docs = get_docs(user_query)

answer = generate_response(user_query, retrieved_docs)
print('Answe is :',answer)

Enter Your Query: what are the GPTs
Answe is : The GPTs are custom versions of ChatGPT. They can be created by users and shared with others. The GPT Store is a platform where users can find and access these custom GPTs.


In [ ]:
user_query = input('Enter Your Query: ')
retrieved_docs = get_docs(user_query)

answer = generate_response(user_query, retrieved_docs)
print('Answe is :',answer)

Enter Your Query: What are the products of Openai
Answe is : The products of OpenAI include:

1. ChatGPT
2. GPT-4
3. GPT-4o Mini
4. DALL·E 3
5. Sora 

Additionally, OpenAI also offers an API for businesses to integrate their models into their own applications.


In [ ]:
user_query = input('Enter Your Query: ')
retrieved_docs = get_docs(user_query)

answer = generate_response(user_query, retrieved_docs)
print('Answe is :',answer)

Enter Your Query: When will be Sora released for the Users to use
Answe is : I am unaware of the Information you are requesting.


In [ ]:
user_query = input('Enter Your Query: ')
retrieved_docs = get_docs(user_query)

answer = generate_response(user_query, retrieved_docs)
print('Answe is :',answer)

Enter Your Query: who are the new members to board of directors
Answe is : The new members to the board of directors are Dr. Sue Desmond-Hellmann, Nicole Seligman, and Fidji Simo


In [ ]:
user_query = input('Enter Your Query: ')
retrieved_docs = get_docs(user_query)

answer = generate_response(user_query, retrieved_docs)
print('Answe is :',answer)

Enter Your Query: Tell me about the Details of partnership announcement made by apple  and how apple is going to use chatGPT 
Answe is : Apple is integrating ChatGPT into experiences within iOS, iPadOS, and macOS, allowing users to access ChatGPT’s capabilities—including image and document understanding—without needing to jump between tools. Siri can also tap into ChatGPT’s intelligence when helpful. Apple users are asked before any questions are sent to ChatGPT, along with any documents or photos, and Siri then presents the answer directly.
